## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Лабораторная работа 3

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [19]:
import pandas as pd
import time

recipes = pd.read_csv('recipes_sample.csv', index_col=0, delimiter=',')
reviews = pd.read_csv('reviews_sample.csv', index_col=0, delimiter=',')
reviews['date'] = pd.to_datetime(reviews['date'])

def meanA(reviews):
    s = 0
    c = 0
    for index, row in reviews.iterrows():
        if row['date'].year == 2010:
            s += row['rating']
            c += 1
    return s / c

def meanB(reviews):
    s = 0
    c = 0
    for index, row in reviews.loc[reviews['date'].dt.year == 2010].iterrows():
        s += row['rating']
        c += 1
    return s / c

def meanC(reviews):
    filtered = reviews.loc[reviews['date'].astype(str).str.contains('2010')]
    return filtered['rating'].mean()

print(meanA(reviews),'\n')
print(meanB(reviews),'\n')
print(meanC(reviews),'\n')

start_time = time.time()
res1 = meanA(reviews)
end_time = time.time()
print(f'meanA: {res1}, time: {end_time - start_time}')

start_time = time.time()
res2 = meanB(reviews)
end_time = time.time()
print(f'meanB: {res2}, time: {end_time - start_time}')

start_time = time.time()
res3 = meanC(reviews)
end_time = time.time()
print(f'meanC: {res3}, time: {end_time - start_time}')

4.4544402182900615 

4.4544402182900615 

4.4544402182900615 

meanA: 4.4544402182900615, time: 3.409162759780884
meanB: 4.4544402182900615, time: 0.32094454765319824
meanC: 4.4544402182900615, time: 0.7417488098144531


In [31]:
# !pip install line_profiler
# %load_ext line_profiler

%lprun -f meanA meanB(reviews)

def meanAB(reviews):
    filtered_reviews = reviews[reviews['date'].dt.year == 2010]
    count = len(filtered_reviews)
    if count > 0:
        return filtered_reviews['rating'].apply(lambda x: float(x)).sum() / count
    else:
        return None

%lprun -f meanAB meanAB(reviews)

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [40]:
%%time

def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

CPU times: total: 0 ns
Wall time: 0 ns


In [36]:
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)